# Getting started with using Doc2Vec ( genism Model )

## Imports

These are the neccessary libs that we are going to use in this session.

In [41]:
import nltk
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

Downloading the neccessay packaages for data-preprocessing

In [2]:
#stopwords like the at a an, unnecesasry
#tokenization into sentences, punkt 
#http://www.nltk.org/
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/buckaroo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/buckaroo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Reading CSV

We are importing the CSV containing the Question , Answering maping. Let's visualize it and check it out

In [6]:
df = pd.read_csv("../data/zeamed-web/zeamed-faq.csv")
df.head(5)

,Question,Answer
0,Does the Zeamed provide 24/7 customer support?,Yes
1,Is it a free service?,yes
2,Why is Zeamed providing a free service?,ZeaMed is absolutely free for consumers.Our a...
3,Where do I find zeamed doctor Profile on this ...,Select the test name and location and click on...
4,Will the patient info be secured?,Yes patient information/data is secured as per...


# Pre-processing

This utility is used for preprocessing the sentences/words that is removing the unneccessary things that doesnot contribute any **meaning or value**

In [9]:
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

This utility is for clean the sentences in the array and append them into the `question_list` array which will use later for training the `Doc2Vec` model

In [22]:
def process_questions(question_list, questions, question_list_name):
# function to transform questions and display progress
    for question in questions:
        question_list.append(review_to_wordlist(question,False))
        if len(question_list) % 5 == 0:
            progress = len(question_list)/len(df) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [25]:
questions = []
process_questions(questions,df.Question,'Questions')
questions

Questions is 22.7% complete.
Questions is 45.5% complete.
Questions is 68.2% complete.
Questions is 90.9% complete.


['doe the zeam provid 24 7 custom support',
 'is it a free servic',
 'whi is zeam provid a free servic',
 'where do i find zeam doctor profil on this app',
 'will the patient info be secur',
 'who are zeam s user',
 'what is zeam',
 'if we book an appoint through you ( zeam com ) do i stand a better chanc of get an appoint',
 'are there ani addit featur i have access to with this app',
 'how do i instal the zeam app for doctor',
 'is it free to use',
 'i book an appoint through you with a doctor will i have to be there on time what if i arriv late',
 'is it a free servic whi is zeam provid a free servic',
 'do i have to show the zeam appoint id when visit a clinic hospit for my appoint',
 'some doctor clinic hospit in my area are not list on zeam com what do i do about it',
 'is zeam safe to approach',
 'what if a patient has ani issu schedul an appoint onlin',
 'how can patient search for appoint',
 'if a patient has an appoint book at a particular time can the patient book anoth appo

`query` is out input and we pre-processing , storing it in a `question` which we will use it for testing the `Doc2Vec` Model.

In [97]:
query="who are you ?"
question = review_to_wordlist(query,False)
question

'who are you'

`labeled_questions` will be an array containing the sentences in form of `LabeledSentence` object , which we'll inject into `Doc2Vec` model.

In [70]:
labeled_questions = []

for i,sentence in enumerate(questions):
    #print(i,sentence)
    labeled_questions.append( LabeledSentence( sentence.split() , [(i+1)] ) )

labeled_questions

/home/buckaroo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


[LabeledSentence(words=['doe', 'the', 'zeam', 'provid', '24', '7', 'custom', 'support'], tags=[1]),
 LabeledSentence(words=['is', 'it', 'a', 'free', 'servic'], tags=[2]),
 LabeledSentence(words=['whi', 'is', 'zeam', 'provid', 'a', 'free', 'servic'], tags=[3]),
 LabeledSentence(words=['where', 'do', 'i', 'find', 'zeam', 'doctor', 'profil', 'on', 'this', 'app'], tags=[4]),
 LabeledSentence(words=['will', 'the', 'patient', 'info', 'be', 'secur'], tags=[5]),
 LabeledSentence(words=['who', 'are', 'zeam', 's', 'user'], tags=[6]),
 LabeledSentence(words=['what', 'is', 'zeam'], tags=[7]),
 LabeledSentence(words=['if', 'we', 'book', 'an', 'appoint', 'through', 'you', '(', 'zeam', 'com', ')', 'do', 'i', 'stand', 'a', 'better', 'chanc', 'of', 'get', 'an', 'appoint'], tags=[8]),
 LabeledSentence(words=['are', 'there', 'ani', 'addit', 'featur', 'i', 'have', 'access', 'to', 'with', 'this', 'app'], tags=[9]),
 LabeledSentence(words=['how', 'do', 'i', 'instal', 'the', 'zeam', 'app', 'for', 'doctor'], 

Converting the `string` into `LabeledSentence` Object for testing purpose

In [82]:
query_label = LabeledSentence(question,[0])

/home/buckaroo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """Entry point for launching an IPython kernel.


# Building
We are building the model with the array `labeled_questions` our list of questionaires , and vocabulary.

In [91]:
# Build the model

model = Doc2Vec(labeled_questions ,vector_size=50, window=2, min_count=1, workers=4)
model.build_vocab(labeled_questions ,update=True)

# Training 

Training the model for `epochs=20` 

In [102]:
# Train the model
# 20 epochs performs a bit better, but timed out when uploading

model.train(labeled_questions, total_examples=model.corpus_count, epochs=20 )

# Testing

We are testing the input `question` with all the list of questions that we have gathered.

In [100]:
for i , ques in enumerate(questions):
    score = model.wv.n_similarity(ques.lower().split() ,question.lower().split() )
    print(i,ques,'=>',round(score,2))

0 doe the zeam provid 24 7 custom support => 0.88
1 is it a free servic => 0.83
2 whi is zeam provid a free servic => 0.85
3 where do i find zeam doctor profil on this app => 0.94
4 will the patient info be secur => 0.93
5 who are zeam s user => 0.97
6 what is zeam => 0.94
7 if we book an appoint through you ( zeam com ) do i stand a better chanc of get an appoint => 0.93
8 are there ani addit featur i have access to with this app => 0.94
9 how do i instal the zeam app for doctor => 0.97
10 is it free to use => 0.87
11 i book an appoint through you with a doctor will i have to be there on time what if i arriv late => 0.98
12 is it a free servic whi is zeam provid a free servic => 0.84
13 do i have to show the zeam appoint id when visit a clinic hospit for my appoint => 0.96
14 some doctor clinic hospit in my area are not list on zeam com what do i do about it => 0.94
15 is zeam safe to approach => 0.93
16 what if a patient has ani issu schedul an appoint onlin => 0.92
17 how can patien

# Reference :

https://rare-technologies.com/doc2vec-tutorial/ <br>
https://radimrehurek.com/gensim/models/doc2vec.html